In [2]:
import cv2
from PIL import Image
import imagehash
import numpy as np
from datetime import timedelta
import statistics

AVG_WINDOW_T = 2

# Function to compute the average hash for a given list of frames
def compute_average_hash(frames):
    total_hashes = 0
    average_hash = None
    
    # Loop through each frame in the list
    for frame in frames:
        # Convert the frame to PIL Image format
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        # Compute the hash for the current frame
        frame_hash = imagehash.average_hash(frame_pil)
        
        # Add the hash to the total
        total_hashes += 1
        
        # Update the average hash
        if average_hash is None:
            average_hash = np.array(frame_hash.hash, dtype=np.float64)
        else:
            average_hash += np.array(frame_hash.hash, dtype=np.float64)
    
    # Compute the average hash
    average_hash /= total_hashes
    
    # Convert the average hash back to imagehash format
    average_hash = imagehash.ImageHash(average_hash.astype(bool))
    
    return average_hash

def get_average_hashes(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Get the frame rate of the video
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS)

    # Calculate the number of frames for `AVG_WINDOW_T` seconds
    num_frames_per_chunk = int(frame_rate * AVG_WINDOW_T)

    # List to store average hashes for each chunk
    average_hashes = []

    # List to temporarily store frames for each chunk
    frames_chunk = []

    # Loop through each frame in the video
    while True:
        # Read the next frame
        ret, frame = video_capture.read()
        
        # If there are no more frames, break out of the loop
        if not ret:
            break
        
        # Add the frame to the current chunk
        frames_chunk.append(frame)
        
        # Check if the chunk is complete
        if len(frames_chunk) == num_frames_per_chunk:
            # Compute the average hash for the current chunk of frames
            chunk_average_hash = compute_average_hash(frames_chunk)
            
            # Append the average hash to the list
            average_hashes.append(chunk_average_hash)
            
            # Clear the frames chunk list for the next chunk
            frames_chunk = []
    
    return average_hashes

def get_lowest_five_indices(array):
    # Use argsort to get the indices that would sort the array
    sorted_indices = np.argsort(array)
    
    # Return the first five indices, which correspond to the lowest values
    return sorted_indices[:5]


"""
# Example usage:
video_path = "./Videos/video15.mp4"
average_hashesDB = get_average_hashes(video_path)
for i, hash_value in enumerate(average_hashesDB):
    start_time = timedelta(seconds=i * 2)
    end_time = timedelta(seconds=(i + 1) * 2)
    print(f"Average Hash for {start_time} to {end_time}: {hash_value}")

"""

average_hashesDB = []
average_hashesDB.append([0])

for vidNum in range(1, 21):
    video_path = "./Videos/video"+ str(vidNum) +".mp4"
    temp_average_hashes = get_average_hashes(video_path)
    average_hashesDB.append(temp_average_hashes)
    

In [3]:
video_path = "./Videos/Query/video20_filtered.mp4"
query_average_hashs = get_average_hashes(video_path)

diffDB = []

for vidNum in range(1, 21):
    allDiff = []
    for qa_hash in query_average_hashs:
        temp = []
        for original_hash in average_hashesDB[vidNum]:
            temp.append(abs(original_hash - qa_hash))
        
        allDiff.append(min(temp))
    diffDB.append(allDiff)


print(diffDB)

meanArr = []
meanArr.append(999.9)
for vidNum in range(1, 21):
    meanArr.append(statistics.mean(diffDB[vidNum-1]))

print(meanArr)    
print(type(meanArr[0]), type(meanArr[1]))
print(get_lowest_five_indices(meanArr))

[[27, 26, 18, 21, 27, 18, 18, 20, 21, 20], [10, 8, 5, 11, 6, 4, 7, 15, 16, 10], [11, 8, 8, 11, 10, 6, 9, 16, 13, 11], [10, 10, 5, 13, 7, 3, 9, 17, 14, 9], [7, 3, 5, 9, 8, 5, 10, 13, 10, 11], [25, 27, 16, 26, 20, 12, 14, 20, 21, 20], [7, 11, 5, 11, 8, 4, 11, 10, 11, 10], [14, 8, 4, 11, 5, 4, 11, 16, 16, 10], [12, 7, 3, 6, 8, 4, 10, 14, 13, 9], [14, 10, 5, 11, 9, 5, 10, 16, 17, 9], [6, 8, 5, 10, 9, 4, 7, 17, 17, 10], [22, 17, 6, 16, 10, 4, 10, 17, 16, 12], [6, 6, 4, 8, 6, 4, 11, 17, 16, 10], [7, 2, 4, 9, 3, 1, 9, 15, 16, 9], [10, 7, 5, 12, 6, 4, 10, 18, 17, 9], [19, 15, 6, 15, 10, 8, 15, 16, 15, 14], [13, 8, 4, 11, 7, 3, 9, 18, 19, 8], [13, 10, 3, 10, 6, 4, 10, 16, 16, 9], [4, 6, 4, 8, 6, 3, 9, 18, 19, 10], [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]]
[999.9, 21.6, 9.2, 10.3, 9.7, 8.1, 20.1, 8.8, 9.9, 8.6, 10.6, 9.3, 13, 8.8, 7.5, 9.8, 13.3, 10, 9.7, 8.7, 0.2]
<class 'float'> <class 'float'>
[20 14  5  9 19]
